In [1]:
from datasets import load_dataset 

dataset = load_dataset("nielsr/cord-layoutlmv3")

Computing checksums of downloaded files. They can be used for integrity verification. You can disable this by passing ignore_verifications=True to load_dataset


Computing checksums:  50%|#####     | 1/2 [00:08<00:08,  8.41s/it]

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Dataset cord-layoutlmv3 downloaded and prepared to C:/Users/Alten Canada/.cache/huggingface/datasets/nielsr___cord-layoutlmv3/cord/1.0.0/581083d58bb5c35fdcfed4f7b36f9a8bf23acfd28e974edc7b8157c972720203. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

# Pre-processing

In [2]:
from transformers import AutoProcessor
# we'll use the Auto API here - it will load LayoutLMv3Processor behind the scenes,
# based on the checkpoint we provide from the hub
processor = AutoProcessor.from_pretrained("microsoft/layoutlmv3-base", apply_ocr=False)

Downloading:   0%|          | 0.00/275 [00:00<?, ?B/s]

C:\Users\Alten Canada\anaconda3\lib\site-packages\huggingface_hub\file_download.py:123: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Alten Canada\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


Downloading:   0%|          | 0.00/1.14k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/856 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

We'll first create `id2label` and `label2id` mappings, useful for inference. Note that `LayoutLMv3ForTokenClassification` (the model we'll use later on) will simply output an integer index for a particular class (for each token), so we still need to map it to an actual class name.

In [3]:
from datasets.features import ClassLabel

features = dataset["train"].features
column_names = dataset["train"].column_names
image_column_name = "image"
text_column_name = "words"
boxes_column_name = "bboxes"
label_column_name = "ner_tags"

# In the event the labels are not a `Sequence[ClassLabel]`, we will need to go through the dataset to get the
# unique labels.
def get_label_list(labels):
    unique_labels = set()
    for label in labels:
        unique_labels = unique_labels | set(label)
    label_list = list(unique_labels)
    label_list.sort()
    return label_list

if isinstance(features[label_column_name].feature, ClassLabel):
    label_list = features[label_column_name].feature.names
    # No need to convert the labels since they are already ints.
    id2label = {k: v for k,v in enumerate(label_list)}
    label2id = {v: k for k,v in enumerate(label_list)}
else:
    label_list = get_label_list(dataset["train"][label_column_name])
    id2label = {k: v for k,v in enumerate(label_list)}
    label2id = {v: k for k,v in enumerate(label_list)}
num_labels = len(label_list)

In [4]:
print(label_list)

['O', 'B-MENU.NM', 'B-MENU.NUM', 'B-MENU.UNITPRICE', 'B-MENU.CNT', 'B-MENU.DISCOUNTPRICE', 'B-MENU.PRICE', 'B-MENU.ITEMSUBTOTAL', 'B-MENU.VATYN', 'B-MENU.ETC', 'B-MENU.SUB_NM', 'B-MENU.SUB_UNITPRICE', 'B-MENU.SUB_CNT', 'B-MENU.SUB_PRICE', 'B-MENU.SUB_ETC', 'B-VOID_MENU.NM', 'B-VOID_MENU.PRICE', 'B-SUB_TOTAL.SUBTOTAL_PRICE', 'B-SUB_TOTAL.DISCOUNT_PRICE', 'B-SUB_TOTAL.SERVICE_PRICE', 'B-SUB_TOTAL.OTHERSVC_PRICE', 'B-SUB_TOTAL.TAX_PRICE', 'B-SUB_TOTAL.ETC', 'B-TOTAL.TOTAL_PRICE', 'B-TOTAL.TOTAL_ETC', 'B-TOTAL.CASHPRICE', 'B-TOTAL.CHANGEPRICE', 'B-TOTAL.CREDITCARDPRICE', 'B-TOTAL.EMONEYPRICE', 'B-TOTAL.MENUTYPE_CNT', 'B-TOTAL.MENUQTY_CNT', 'I-MENU.NM', 'I-MENU.NUM', 'I-MENU.UNITPRICE', 'I-MENU.CNT', 'I-MENU.DISCOUNTPRICE', 'I-MENU.PRICE', 'I-MENU.ITEMSUBTOTAL', 'I-MENU.VATYN', 'I-MENU.ETC', 'I-MENU.SUB_NM', 'I-MENU.SUB_UNITPRICE', 'I-MENU.SUB_CNT', 'I-MENU.SUB_PRICE', 'I-MENU.SUB_ETC', 'I-VOID_MENU.NM', 'I-VOID_MENU.PRICE', 'I-SUB_TOTAL.SUBTOTAL_PRICE', 'I-SUB_TOTAL.DISCOUNT_PRICE', 'I-SUB

In [5]:
print(id2label)

{0: 'O', 1: 'B-MENU.NM', 2: 'B-MENU.NUM', 3: 'B-MENU.UNITPRICE', 4: 'B-MENU.CNT', 5: 'B-MENU.DISCOUNTPRICE', 6: 'B-MENU.PRICE', 7: 'B-MENU.ITEMSUBTOTAL', 8: 'B-MENU.VATYN', 9: 'B-MENU.ETC', 10: 'B-MENU.SUB_NM', 11: 'B-MENU.SUB_UNITPRICE', 12: 'B-MENU.SUB_CNT', 13: 'B-MENU.SUB_PRICE', 14: 'B-MENU.SUB_ETC', 15: 'B-VOID_MENU.NM', 16: 'B-VOID_MENU.PRICE', 17: 'B-SUB_TOTAL.SUBTOTAL_PRICE', 18: 'B-SUB_TOTAL.DISCOUNT_PRICE', 19: 'B-SUB_TOTAL.SERVICE_PRICE', 20: 'B-SUB_TOTAL.OTHERSVC_PRICE', 21: 'B-SUB_TOTAL.TAX_PRICE', 22: 'B-SUB_TOTAL.ETC', 23: 'B-TOTAL.TOTAL_PRICE', 24: 'B-TOTAL.TOTAL_ETC', 25: 'B-TOTAL.CASHPRICE', 26: 'B-TOTAL.CHANGEPRICE', 27: 'B-TOTAL.CREDITCARDPRICE', 28: 'B-TOTAL.EMONEYPRICE', 29: 'B-TOTAL.MENUTYPE_CNT', 30: 'B-TOTAL.MENUQTY_CNT', 31: 'I-MENU.NM', 32: 'I-MENU.NUM', 33: 'I-MENU.UNITPRICE', 34: 'I-MENU.CNT', 35: 'I-MENU.DISCOUNTPRICE', 36: 'I-MENU.PRICE', 37: 'I-MENU.ITEMSUBTOTAL', 38: 'I-MENU.VATYN', 39: 'I-MENU.ETC', 40: 'I-MENU.SUB_NM', 41: 'I-MENU.SUB_UNITPRICE', 42:

In [6]:
def prepare_examples(examples):
    images = examples[image_column_name] ##if you use an image path, this will need to be updated to read the image in
    words = examples[text_column_name]
    boxes = examples[boxes_column_name]
    word_labels = examples[label_column_name]

    encoding = processor(images, words, boxes=boxes, word_labels=word_labels,
                       truncation=True, padding="max_length")

    return encoding 


In [7]:
from datasets import Features, Sequence, ClassLabel, Value, Array2D, Array3D

# we need to define custom features for `set_format` (used later on) to work properly
features = Features({
    'pixel_values': Array3D(dtype="float32", shape=(3, 224, 224)),
    'input_ids': Sequence(feature=Value(dtype='int64')),
    'attention_mask': Sequence(Value(dtype='int64')),
    'bbox': Array2D(dtype="int64", shape=(512, 4)),
    'labels': Sequence(feature=Value(dtype='int64')),
})



Apply to train/test datasets

In [9]:
train_sample = dataset["train"]
train_dataset = train_sample.map(
    prepare_examples,
    batched=True,
    remove_columns=column_names,
    features=features,
)

eval_dataset = dataset["test"].map(
    prepare_examples,
    batched=True,
    remove_columns=column_names,
    features=features,
)

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [10]:
train_dataset

Dataset({
    features: ['pixel_values', 'input_ids', 'attention_mask', 'bbox', 'labels'],
    num_rows: 800
})

In [11]:
example = train_dataset[0]
processor.tokenizer.decode(example["input_ids"])

'<s> x x x x x x x x x x x x x x x x x x x x x x 1 Nasi Campur Bali 75,000 1 Bbk Bengil Nasi 125,000 1 MilkShake Starwb 37,000 1 Ice Lemon Tea 24,000 1 Nasi Ayam Dewata 70,000 3 Free Ice Tea 0 1 Organic Green Sa 65,000 1 Ice Tea 18,000 1 Ice Orange 29,000 1 Ayam Suir Bali 85,000 2 Tahu Goreng 36,000 2 Tempe Goreng 36,000 1 Tahu Telor Asin 40,000. 1 Nasi Goreng Samb 70,000 3 Bbk Panggang Sam 366,000 1 Ayam Sambal Hija 92,000 2 Hot Tea 44,000 1 Ice Kopi 32,000 1 Tahu Telor Asin 40,000 1 Free Ice Tea 0 1 Bebek Street 44,000 1 Ice Tea Tawar 18,000 Sub-Total 1,346,000 Service 100,950 PB1 144,695 Rounding -45 Grand Total 1,591,600</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad

Convert to Torch

In [12]:
train_dataset.set_format("torch")

Verify conversion format

In [13]:
import torch

example = train_dataset[0]
for k,v in example.items():
    print(k,v.shape)

pixel_values torch.Size([3, 224, 224])
input_ids torch.Size([512])
attention_mask torch.Size([512])
bbox torch.Size([512, 4])
labels torch.Size([512])


# Metrics

In [15]:
from datasets import load_metric

metric = load_metric("seqeval")

In [16]:
import numpy as np

return_entity_level_metrics = False

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    if return_entity_level_metrics:
        # Unpack nested dictionaries
        final_results = {}
        for key, value in results.items():
            if isinstance(value, dict):
                for n, v in value.items():
                    final_results[f"{key}_{n}"] = v
            else:
                final_results[key] = value
        return final_results
    else:
        return {
            "precision": results["overall_precision"],
            "recall": results["overall_recall"],
            "f1": results["overall_f1"],
            "accuracy": results["overall_accuracy"],
        }

# Model

Define model

In [17]:
from transformers import LayoutLMv3ForTokenClassification

model = LayoutLMv3ForTokenClassification.from_pretrained(
    "microsoft/layoutlmv3-base",
    id2label=id2label,
    label2id=label2id
)

Downloading:   0%|          | 0.00/501M [00:00<?, ?B/s]

Some weights of LayoutLMv3ForTokenClassification were not initialized from the model checkpoint at microsoft/layoutlmv3-base and are newly initialized: ['classifier.dense.weight', 'classifier.out_proj.weight', 'classifier.dense.bias', 'classifier.out_proj.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## Define training arguments and trainer

Next we define the `TrainingArguments`, which define all hyperparameters related to training. Note that there is a huge amount of parameters to tweak, check the [docs](https://huggingface.co/docs/transformers/main_classes/trainer#transformers.TrainingArguments) for more info.
The training arguments I set here:
- max steps for how long I thought it would take to train - you may need more steps
- batch_size was tweaked based on GPU memory
- push to hub was set to save my model to the hugging face hub
- evaluation strategy, steps was set to periodically evaluate the model
- metric for best model was to save the best checkpoint model based on accuracy

In [33]:
from transformers import TrainingArguments, Trainer
import os
os.makedirs("layoutlmv3-finetuned-cord_100", exist_ok=True)

training_args = TrainingArguments(output_dir="layoutlmv3-finetuned-cord_100",
                                  overwrite_output_dir=True,
                                  num_train_epochs=100,
                                  logging_strategy="steps",
                                  save_strategy="steps",
                                  per_device_train_batch_size=5,
                                  per_device_eval_batch_size=5,
                                  push_to_hub=False,  # after training, we won't push our model to the hub
                                  learning_rate=1e-5,
                                  evaluation_strategy="steps",
                                  eval_steps=100,
                                  load_best_model_at_end=True,
                                  metric_for_best_model="accuracy")

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [34]:
from transformers import EarlyStoppingCallback

In [35]:
earlystop_callback = EarlyStoppingCallback(
    early_stopping_patience=5,
    early_stopping_threshold=1
)
callbacks = [earlystop_callback]

We can now instantiate a Trainer, with the model and args defined above. We also provide our datasets, as well as a "default data collator" - which will batch the examples using `torch.stack`. We also provide our `compute_metrics` function defined above.

In [36]:
from transformers.data.data_collator import default_data_collator

# Initialize our Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=processor,
    data_collator=default_data_collator,
    compute_metrics=compute_metrics,
    callbacks=callbacks
)

# Training

In [37]:
trainer.train()

***** Running training *****
  Num examples = 800
  Num Epochs = 100
  Instantaneous batch size per device = 5
  Total train batch size (w. parallel, distributed & accumulation) = 5
  Gradient Accumulation steps = 1
  Total optimization steps = 16000
  Number of trainable parameters = 125964477


Step,Training Loss,Validation Loss


RuntimeError: [enforce fail at C:\b\abs_f0dma8qm3d\croot\pytorch_1669187301762\work\c10\core\impl\alloc_cpu.cpp:81] data. DefaultCPUAllocator: not enough memory: you tried to allocate 120643440 bytes.

In [ ]:
trainer.save_model("./layoutlmv3-finetuned-cord_100/model_save_archive")